In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import warnings
warnings.filterwarnings("error")

In [9]:
from __future__ import division
import sys
import math
import numpy as np
from numpy.linalg import norm
import random
import time
from imagestack import *
#https://www.rcsb.org/structure/1DPX
#STRUCTURE OF HEN EGG-WHITE LYSOZYME
#fcxi = "/reg/d/psdm/cxi/cxitut13/res/iris/psocake/r0010/cxitut13_0010.cxi"
fcxi = "/reg/d/psdm/cxi/cxitut13/scratch/iris/testing.cxi"

ch2stkx,ch2stky,ch2stkz = mapcheetch2stack(experimentName="cxitut13",detInfo="DscCsPad")
print (type(ch2stkx))
print (type(ch2stky))
print (type(ch2stkz))
xraw,yraw = getBragg4pcxi(fcxi,pidx=None,eventInd=78)

peaks = np.zeros((len(xraw),3))
peaks[:,0] = ch2stkx[xraw,yraw]
peaks[:,1] = ch2stky[xraw,yraw]
peaks[:,2] = ch2stkz[xraw,yraw]


# peaks -- index (1st: the nth panel; 2nd: x-position on the nth panel; 3rd: y-position on the nth panel)
# iX, iY -- the real position

iX = np.load('cxitut13_iX.npy') 
iY = np.load('cxitut13_iY.npy')
print(np.shape(iX)) # total 32 panels

cx = 877 # self.parent.cx # center along the x-axis
cy = 864 # self.parent.cy # center along the y-axis

photonEnergy = 8.06 # [keV]
wavelength = 12.407002 / float(photonEnergy) * 1e-10  # [m]

D = 0.158 # detector distance [m]
pixSize = 110e-6 # [m]

cenX = iX[np.array(peaks[:, 0], np.dtype(float)),
          np.array(peaks[:, 1], np.dtype(float)),
          np.array(peaks[:, 2], np.dtype(float))] + 0.5 # pixels

cenY = iY[np.array(peaks[:, 0], np.dtype(float)),
          np.array(peaks[:, 1], np.dtype(float)),
          np.array(peaks[:, 2], np.dtype(float))] + 0.5 # pixels

# eqn(3) in "Steller et al."
Xd = (cenX - cx) * pixSize # [m]
Yd = (cenY - cy) * pixSize # [m]
r = np.sqrt(Xd**2 + Yd**2 + D**2) # [m]
reflections = np.array([Xd/r, Yd/r, D/r-1]) / (wavelength) # [m^-1]
reflections = reflections.T * 1.0e-10 # zhen
N_reflections = len(reflections) # zhen

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
(32, 185, 388)


IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
# from __future__ import division
import sys
import math
import numpy as np
from numpy.linalg import norm
import random
#from sklearn.linear_model import LinearRegression

# Possible direct vector
class tvector:
    def __init__(self, *args):
        if len(args) == 1:
            if isinstance(args[0], int):
                self.tvector_new(args[0])
                return
            else:
                raise ValueError("I want integer!!")
        else:
            ValueError("I don't know how to deal with that many parameters.")
    def tvector_new(self, n):
        self.t = np.zeros(3) # zhen
        self.n = 0 # number of fitting reflections
        self.fits = np.zeros(n)  # label of fitted reflections


class asdf_cell:
    def __init__(self, *args):
        if len(args) == 1:
            if isinstance(args[0], int):
                self.asdf_cell_new(args[0])
                return
            else:
                raise ValueError("I want integer!!")
        else:
            ValueError("I don't know how to deal with that many parameters.")
    def asdf_cell_new(self, N_reflections=0): # zhen
        self.axes = np.zeros((3,3)) #zhen
        self.reciprocal = np.zeros((3,3)) #zhen
        self.n = 0 # number of fitting reflections
        self.volume = None
        self.N_refls = N_reflections # zhen # total number of reflections
        self.reflections = np.zeros(self.N_refls) # zhen # reflections[i] = 1 if reflections fits
        self.indices = np.zeros((self.N_refls,3)) # zhen # indices[i] = [h, k, l] for all reflections (not rounded)
        self.acl = 0 # zhen #minimum number of reflections fitting to one of the axes[]
        self.n_max = 0 # zhen # maximum number of reflections fitting to some t-vector
        self.dist = 0


def dot_product(vec1,vec2):
    return np.dot(vec1,vec2)

def cross_product(vec1,vec2):
    return np.cross(vec1,vec2)

def calc_volume(vec1,vec2,vec3):
    return np.cross(vec1,vec2).dot(vec3)


def check_cell_angles(veca,vecb,vecc,max_cos):
    """
    check whether three vectors can build a fine cell, return 0 if cos(theta) larger than max_cos
    """
    a = norm(veca)
    b = norm(vecb)
    c = norm(vecc)

    ab = dot_product(veca,vecb)
    bc = dot_product(vecb,vecc)
    ca = dot_product(vecc,veca)

    cosa = bc/b/c
    cosb = ca/a/c
    cosg = ab/a/b

    if np.abs(cosa) > max_cos or np.abs(cosb) > max_cos or np.abs(cosg) > max_cos:
        return 0
    return 1

def calc_reciprocal(direct):
    reciprocal = np.zeros((3,3))
    reciprocal[0] = cross_product(direct[1],direct[2])
    volume = dot_product(direct[0],reciprocal[0])
    # print direct, reciprocal, volume
    reciprocal[0] = reciprocal[0]/volume
    reciprocal[1] = cross_product(direct[2],direct[0])
    reciprocal[1] = reciprocal[1]/volume
    reciprocal[2] = cross_product(direct[0],direct[1])
    reciprocal[2] = reciprocal[2]/volume
    return reciprocal


def refine_asdf_cell(cell, reflections, N_reflections, IndexFit): # zhen 
    index = np.where(cell.reflections == 1)
    X = np.around(cell.indices[index]).copy()
    r = reflections[index].copy()
    n = len(index[0])
    A_prime_fit = np.linalg.lstsq(r,X)[0].T
    cell.axes = A_prime_fit.copy()
    cell.reciprocal = calc_reciprocal(cell.axes)
    a = np.sqrt(np.sum(cell.axes**2, axis=1))
    if (np.abs(a) > 10000).any():
        return 0
    return 1


def find_acl(t1,t2,t3):
    return min(t1.n, t2.n, t3.n)

#     i = t1.n
#     j = t2.n
#     k = t3.n
#     if i <= j and i <= k:
#         return i
#     if j <= i and j <= k:
#         return j
#     if k <= i and k <= j:
#         return k
#     print("This point never reached!")



def create_cell(tvec1,tvec2,tvec3,cell,IndexFit,volume_min,volume_max,reflections,N_reflections):
    # calculate the volume formed by the T-triplet, compare with volume_min & volume_max
    cell.axes[0] = tvec1.t.copy()
    cell.axes[1] = tvec2.t.copy()
    cell.axes[2] = tvec3.t.copy()
    volume = calc_volume(tvec1.t,tvec2.t,tvec3.t)
    if np.abs(volume) < volume_min or np.abs(volume) > volume_max:
        return 0

    cell.volume = volume
    check_refl_fitting_cell(cell,reflections,N_reflections,IndexFit) # zhen

    if cell.n < 6:
        return 0
    #print "reduce_asdf_cell_before", c.axes[0],c.axes[1],c.axes[2]
    reduce_asdf_cell(cell) # zhen
    #print "reduce_asdf_cell_after", c.axes[0],c.axes[1],c.axes[2]
    # If one of the cell angles > 135 or < 45 return 0
    if not check_cell_angles(tvec1.t,tvec2.t,tvec3.t,0.71):
        return 0

    # Index reflections with new cell axes
    check_refl_fitting_cell(cell,reflections,N_reflections,IndexFit) # zhen

    # Refine cell until the number of fitting reflections stops increasing
    n = 0
    cell_correct = 1
    while cell.n - n > 0 and cell_correct:
        n = cell.n
        cell_correct = refine_asdf_cell(cell,reflections,N_reflections,IndexFit)
        check_refl_fitting_cell(cell,reflections,N_reflections,IndexFit)  # zhen

    return cell_correct

def asdf_cell_memcpy(result, cell):
    result.axes = cell.axes.copy()
    result.reciprocal = cell.reciprocal.copy()
    result.n = cell.n
    result.volume = cell.volume
    result.N_refls = cell.N_refls # zhen # total number of reflections
    result.reflections = cell.reflections.copy() # zhen # reflections[i] = 1 if reflections fits
    result.indices = cell.indices.copy() # zhen # indices[i] = [h, k, l] for all reflections (not rounded)
    result.acl = cell.acl # zhen #minimum number of reflections fitting to one of the axes[]
    result.n_max = cell.n_max # zhen # maximum number of reflections fitting to some t-vector
    result.dist = cell.dist

def find_cell(tvectors,N_tvectors,IndexFit,volume_min,volume_max,n_max,reflections,N_reflections,result):
    # Only tvectors with the number of fitting reflections > acl are considered
    if N_reflections < 18:
        acl = 6
    else:
        acl = N_reflections/3

    cell = asdf_cell(N_reflections) # zhen
    
    # Traversing a 3d array in slices perpendicular to the main diagonal
    for sl in range (3*N_tvectors):
        if sl < 2*N_tvectors:
            i_min = 0
        else:
            i_min = sl-2*N_tvectors

        if sl < N_tvectors:
            i_max =  sl
        else:
            i_max = N_tvectors

        for i in range(i_min,i_max):
            # print("start/i_min/i_max/sl = ", i, i_min, i_max, sl)
            if tvectors[i].n > acl:
                if sl-N_tvectors-2*i-1 < 0:
                    j_min = i+1
                else:
                    j_min = sl-N_tvectors-i

                if sl-N_tvectors-i < 0:
                    j_max = sl-i
                else:
                    j_max = N_tvectors
            
                for j in range(j_min,j_max ):
                    if tvectors[j].n > acl:
                        k = sl-i-j-1
                        # print (i,j,k)
                        if k > j and tvectors[k].n > acl and check_cell_angles(tvectors[i].t,tvectors[j].t,tvectors[k].t,0.99):
                            # print(i,j,k)
                            if not create_cell(tvectors[i],tvectors[j],tvectors[k],cell,IndexFit,volume_min,volume_max,reflections,N_reflections):
                                break
                            
                            # print(i,j,k)
                            acl = find_acl(tvectors[i],tvectors[j],tvectors[k])
                            cell.acl = acl
                            cell.n_max = n_max
                            
                            print(get_lattice(tvectors[i].t, tvectors[j].t, tvectors[k].t), cell.acl, cell.n)
                            
                            reduce_asdf_cell(cell);

                        # If the new cell has more fitting
                        # reflections save it to result
                            if result.n < cell.n:
                                asdf_cell_memcpy(result, cell)
                                # result = copy.deepcopy(c)

                            acl+=1
                            
                            
                            print(get_lattice(tvectors[i].t, tvectors[j].t, tvectors[k].t), cell.acl, cell.n)

                            if acl > n_max: 
                                # print(111)
                                break
                            if tvectors[j].n <= acl or tvectors[i].n <= acl:
                                # print(222)
                                break

                if acl > n_max: # zhen BACK A TAB
                    # print(333)
                    break
                if tvectors[i].n <= acl: # zhen # BACK A TAB
                    # print(444)
                    break

        if acl > n_max: # zhen # BACK A TAB
            # print(555)
            break
                
    if result.n:
        return 1
    return 0



# Generate triplets of integers < N_reflections in random sequence
#def generate_triplets(N_reflections,N_triplets_max): # zhen

#     # Number of triplets = c_n^3 if n - number of reflections
#     N_triplets = N_reflections*(N_reflections-1)*(N_reflections-2)/6

#     if N_triplets > N_triplets_max or N_reflections > 1000:
#         N_triplets = N_triplets_max

#     # N = N_triplets  # zhen

#     list_2d = []
#     i = 0 # zhen
#     while i < N_triplets: # zhen
#         # Generate three different integer numbers < N_reflections 
#         sub = np.array(random.sample(range(0,N_reflections),3)) # This can avoid replacement, and already int
#         # Sort (i, j, k)
#         sub.sort()

#         # Check if it's already in triplets[]
#         is_in_triplets = False
#         for j in range(i):
#             if (list_2d[j] == sub).all():
#                 is_in_triplets = True
#                 break
#         if is_in_triplets == False:
#             i += 1 # zhen
#             list_2d.append(sub)
#         triplets = np.array(list_2d)
#     return triplets 


def generate_triplets(N_reflections,N_triplets_max):

    # Number of triplets = c_n^3 if n - number of reflections
    N_triplets = N_reflections*(N_reflections-1)*(N_reflections-2)/6

    if N_triplets > N_triplets_max or N_reflections > 1000:
        N_triplets = N_triplets_max
    
    triplets = np.zeros((N_triplets,3)).astype(int)
    
    temp = np.array(random.sample(range(0,N_reflections**3),N_triplets)).astype(int)
    triplets[:,2] = temp % N_reflections
    triplets[:,0] = temp / N_reflections**2
    triplets[:,1] = (temp - triplets[:,0]*N_reflections**2 - triplets[:,2])/ N_reflections
    return triplets


# Calculates normal to a triplet vec1, vec2, vec3. Returns 0 if reflections are on the same line
def calc_normal(vec1,vec2,vec3):
    c12 = np.cross(vec1,vec2)
    c23 = np.cross(vec2,vec3)
    c31 = np.cross(vec3,vec1) # zhen
#     res = np.zeros(3) # zhen
    res = c12 + c23 + c31
#     for i in range(3):
#         res[i] = c12[i]+c23[i]+c31[i]
    if norm(res) < 0.0001:
        return 0,0 # zhen
    else:
        res = res/norm(res)
        normal = res
    return 1,normal # zhen


def vecFFT(arr, N):
    kn = np.meshgrid(arr, np.arange(N),indexing="ij") 
    return np.ones(len(arr)).dot( np.exp( -1.j * 2. * np.pi * kn[0] * kn[1] / N ))


def find_ds_fft(projections,N_projections,d_max):

    n = N_projections
    projections_sorted = np.sort(projections)

    N = 1024 # zhen, check the value of fftw in asdf.c
    
    """
    #for i in range(N): # zhen
    inp = np.zeros(N) # zhen

    for i in range(n):
        k = ((projections_sorted[i]-projections_sorted[0])/(projections_sorted[n-1]-projections_sorted[0])*(N-1)).astype(int) # zhen
        if k>=N or k<0:
            return -1
        inp[k]+=1 # zhen

    """
    
    out = vecFFT((projections_sorted-projections_sorted[0])/(projections_sorted[n-1]-projections_sorted[0])*(N-1), N)
    out = np.abs(out)
    # out = np.abs(np.fft.fft(inp)) # zhen
    #print out.shape

    i_max = int(d_max*(projections_sorted[n-1]-projections_sorted[0])) # zhen
    # print i_max

#     d = 1
#     maxi = 0
#     for i in range(1,i_max+1):
#         # a = np.sqrt(out[i][0]*out[i][0]+out[i][1]*out[i][1]) # zhen [0],[1] is the complex number in C
#         a = out[i] # zhen
#         if a > maxi:
#             maxi = a
#             d = i
    d = np.argmax(out[1:i_max+1]) + 1
    ds = (projections_sorted[n-1]-projections_sorted[0])/d
    return ds




# Returns number of reflections fitting ds
# A projected reflection fits a one-dimensional lattice with elementary lattice vector d* 
# if its absolute distance to the nearest lattice point is less than LevelFit.
def check_refl_fitting_ds(projections,N_projections,ds,LevelFit):
    if ds == 0:
        return 0
#     n = 0
#     for i in range(N_projections):
#         if np.abs(projections[i]-ds*round(projections[i]/ds)) < LevelFit:
#             n+=1
    return np.sum(np.abs(projections-ds*np.around(projections/ds)) < LevelFit)



# Refines d*, writes 1 to fits[i] if the i-th projection fits d*
def refine_ds(projections,N_projections,ds,LevelFit,fits):
    N_fits = 0
    N_new = N_projections
    ds_new = ds

#     indices = np.zeros(N_projections) # zhen
#     fit_refls = np.zeros(N_projections)  # zhen, may be put inside the for loop

    while N_fits < N_new:
        N_fits = 0
        # print N_projections, N_new, N_fits
#         for i in range(N_projections):
#             if np.abs(projections[i]-ds_new*round(projections[i]/ds_new)) < LevelFit:
#                 fit_refls[N_fits] = projections[i]
#                 indices[N_fits] = round(projections[i]/ds_new)
#                 N_fits += 1
#                 fits[i] = 1
#             else:
#                 fits[i] = 0

        index = np.where(np.abs(projections-ds_new*np.around(projections/ds_new)) < LevelFit)
        fit_refls = projections[index].copy()
        indices = np.around(projections[index]/ds_new)
        N_fits = len(index[0])
        fits[index] = 1
        index = np.where(np.abs(projections-ds_new*np.around(projections/ds_new)) >= LevelFit)
        fits[index] = 0

        # linear fitting without a constant term
        if N_fits == 0:
            return ds_new
        
        ## tmp remove # zhen
#         model = sm.OLS(fit_refls[:N_fits], indices[:N_fits]) # zhen
#         results = model.fit()
#         c1 = results.params[0]
        ###### tmp remove # zhen 
        c1 = np.linalg.lstsq(indices.reshape((N_fits,1)), fit_refls)[0][0] 

        N_new = check_refl_fitting_ds(projections,N_projections,c1,LevelFit)
        # print "c1/ds = ", c1, ds, N_fits, N_new
        
        if N_new >= N_fits:
            ds_new = c1

    return ds_new




def check_refl_fitting_cell(cell,reflections,N_reflections,IndexFit):
    cell.reciprocal = calc_reciprocal(cell.axes)
    cell.n = 0 # zhen
#     dist = np.zeros(3) # zhen 
#     cell.dist = 0
#     for i in range(N_reflections):
#         for j in range(3):
#             dist[j] = 0
#         for j in range(3):
#             cell.indices[i,j] = dot_product(reflections[i],cell.axes[j]) # zhen 
#             for k in range(3):
#                 dist[k] += cell.reciprocal[j][k]*(cell.indices[i,j] - round(cell.indices[i,j]))
#         # A reflection fits if the distance (in reciprocal space) between the observed and calculated reflection position is less than Indexfit
        
#         cell.dist += dist[0]*dist[0]+dist[1]*dist[1]+dist[2]*dist[2]
        
#         if dist[0]*dist[0]+dist[1]*dist[1]+dist[2]*dist[2] < IndexFit*IndexFit:
#             cell.reflections[i] = 1 # zhen
#             cell.n += 1
            
#         else:
#             cell.reflections[i] = 0 # zhen 

    cell.indices = reflections.dot(cell.axes.T)
    dist = (cell.indices - np.around(cell.indices)).dot(cell.reciprocal)
    index = np.where(np.sum(dist**2,axis=1)<IndexFit*IndexFit)
    cell.reflections[index] = 1
    cell.n = len(index[0])
    index = np.where(np.sum(dist**2,axis=1)>=IndexFit*IndexFit)
    cell.reflections[index] = 0
    
    
    
    # print "check_refl = ", c.n, c.reciprocal*10
    return 1


def calc_length(v): # zhen
    return norm(v)

def calc_angle(v1, v2): # zhen
    l1 = calc_length(v1)
    l2 = calc_length(v2)
    return np.arccos(v1.dot(v2)/l1/l2) * 180.0 / np.pi

def reduce_asdf_cell(cell): # zhen

    changed = 1
    n = 0
    while changed:
        n += 1
        changed = 0
        
        v1 = cell.axes[0].copy() # zhen
        v2 = cell.axes[1].copy() # zhen 
        v3 = cell.axes[2].copy() # zhen 

        a = calc_length(v1)
        b = calc_length(v2)
        c = calc_length(v3)
        
        gamma = calc_angle(v1,v2)
        alpha = calc_angle(v2,v3)
        beta = calc_angle(v3,v1)
        
        if changed == 0:
            if gamma < 90:
                v2 = -v2
                gamma = 180-gamma
                alpha = 180-alpha      
            v2 = v2+v1
            bb = calc_length(v2)
            if bb < b:
                b = bb
                if a < b:
                    cell.axes[1] = v2.copy()
                else:
                    cell.axes[0] = v2.copy()
                    cell.axes[1] = v1.copy()
                changed = 1
        
        if changed == 0:
            if beta < 90:  # zhen
                v3 = -v3
                beta = 180-beta
                alpha = 180-alpha
            v3 = v3+v1
            cc = calc_length(v3)
            if cc < c:
                c = cc
                if b < c:
                    cell.axes[2] = v3.copy()
                elif a < c:
                    cell.axes[1] = v3.copy()
                    cell.axes[2] = v2.copy()
                else:
                    cell.axes[0] = v3.copy()
                    cell.axes[1] = v1.copy()
                    cell.axes[2] = v2.copy()
                changed = 1
                
        if changed == 0:
            if alpha < 90:
                v3 = -v3 # zhen
                beta = 180 - beta
                alpha = 180 - alpha
                
            v3 = v3+v2
            cc = calc_length(v3)
            if cc < c:
                c = cc
                if b < c:
                    cell.axes[2] = v3.copy()
                elif a < c:
                    cell.axes[1] = v3.copy()
                    cell.axes[2] = v2.copy()
                else:
                    cell.axes[0] = v3.copy()
                    cell.axes[1] = v1.copy()
                    cell.axes[2] = v2.copy()
                changed = 1

        if n > 30:
            changed = 0
        
    # zhen # BACK A TAB
    cell.volume = calc_volume(cell.axes[0],cell.axes[1],cell.axes[2]) # zhen
    if cell.volume < 0:
        cell.axes[2] = -cell.axes[2].copy()
        cell.volume = -1.0 * cell.volume      
        cell.reciprocal = calc_reciprocal(cell.axes)
    
    return 1




def index_refls(reflections,N_reflections,d_max,volume_min,volume_max,LevelFit,IndexFit,N_triplets_max,cell):
    # N_triplets = 0 # zhen
    triplets = generate_triplets(N_reflections,N_triplets_max) # zhen
    fits = np.zeros(N_reflections).astype(int) # zhen
    N_triplets = len(triplets) # zhen
    if N_triplets == 0:
        return 0

    tvectors = [None]*N_triplets # zhen

    N_tvectors = 0

    n_max = 0 # maximum number of reflections fitting one of tvectors

    for i in range(N_triplets):
        hasnorm, normal = calc_normal(reflections[triplets[i][0]],reflections[triplets[i][1]],reflections[triplets[i][2]]) # zhen
        if hasnorm: # zhen 
            # Calculate projections of reflections to normal
            projections = np.zeros(N_reflections) # zhen
            for k in range(N_reflections):
                projections[k] = dot_product(normal,reflections[k])  # normal not defined

            # Find ds - period in 1d lattice of projections
            ds = find_ds_fft(projections,N_reflections,d_max)   
            if ds < 0.0:
                print ("find_ds_fft() failed.\n")
                return 0

            # Refine ds, write 1 to fits[i] if reflections[i] fits ds
            ds = refine_ds(projections,N_reflections,ds,LevelFit,fits)

            # n - number of reflections fitting ds
            n = check_refl_fitting_ds(projections,N_reflections,ds,LevelFit)

            # normal/ds - possible direct vector
            normal = normal/ds

            # print "N/N_tvectors/n_fits = ", i, N_tvectors, n
            
            if n > N_reflections/3 and n > 6:

                tvectors[N_tvectors] = tvector(N_reflections) # zhen

                tvectors[N_tvectors].t = normal
                tvectors[N_tvectors].fits = fits

                tvectors[N_tvectors].n = n

                N_tvectors+=1

                if n > n_max:
                    n_max = n
            
                # print "CORRECT: N/N_tvectors/n_fits = ", i, N_tvectors, n
 

        if (i != 0 and i % 10000 == 0) or i == N_triplets - 1:

            # Sort tvectors by length
            def sqdist(vector):
                if vector is None:  # zhen
                    return 1e9
                return sum(x*x for x in vector.t)

            # print "vector.t = ", tvectors[0].t
            tvectors.sort(key=sqdist)  

            # Three shortest independent tvectors with t.n > acl
            # determine the final cell. acl is selected for the
            # solution with the maximum number of fitting
            # reflections
            
            print ("Number of ts = ", N_tvectors)
            # print "Tvectors", tvectors
            
#             for idx in range(min(100, len(tvectors))):
#                 if tvectors[idx] is not None:
#                     print (tvectors[idx].t, calc_length(tvectors[idx].t), tvectors[idx].n)

            find_cell(tvectors,N_tvectors,IndexFit,volume_min,
                      volume_max,n_max,reflections,
                      N_reflections,cell)

            if cell.n > 4*n_max/5:
                break

    if cell.n:
        return 1
    return 0                   


def get_lattice(t1, t2, t3):
    return calc_length(np.array(t1)),calc_length(np.array(t2)),calc_length(np.array(t3)),\
                calc_angle(np.array(t2),np.array(t3)), calc_angle(np.array(t1),np.array(t3)), calc_angle(np.array(t1),np.array(t2))  

def run_asdf():
    np.random.seed(1234)
    random.seed(1234)
    print ("hello")
    LevelFit = 1./1000
    IndexFit = 1./500
    d_max = 1000. #  thrice the maximum expected axis length
    volume_min = 100.  ## in A^3
    volume_max = 100000000.  ## in A^3

    N_triplets_max = 10000 # maximum number of triplets

    cell = asdf_cell(N_reflections)
    if cell is None: # zhen, you shouldn't use == on None
        print ("Failed to allocate asdf_cell in run_asdf!\n")
        return 0
    
    if N_reflections == 0:
        return 0

    # random.seed(5643)
    j = index_refls(reflections,N_reflections,d_max,volume_min,volume_max,LevelFit,IndexFit,N_triplets_max,cell)
    
    print ("j=", j)
    
    if j:
        print ("\n")
        print ('astar(nm^-1) =', cell.reciprocal[0]*10)
        print ('bstar(nm^-1) =', cell.reciprocal[1]*10)
        print ('cstar(nm^-1) =', cell.reciprocal[2]*10, "\n")
        
        
        print ('a(A) =', cell.axes[0])
        print ('b(A) =', cell.axes[1])
        print ('c(A) =', cell.axes[2], "\n")
        
        print ("Lattice = ", round(calc_length(cell.axes[0]),2), \
                             round(calc_length(cell.axes[1]),2), \
                             round(calc_length(cell.axes[2]),2), \
                             round(calc_angle(cell.axes[1],cell.axes[2]),2), round(calc_angle(cell.axes[0],cell.axes[2]),2), round(calc_angle(cell.axes[1],cell.axes[0]),2) )
        
        print ("Fitted N = ", cell.n )
        
        return cell

    return cell

In [ ]:
import time
tic = time.time()
cell = run_asdf()
print (time.time() - tic)

In [ ]:
print np.sum(((reflections.dot(cell.axes.T) - np.around(reflections.dot(cell.axes.T))).dot(cell.reciprocal))**2)
print cell.dist
(reflections.dot(cell.axes.T) - np.around(reflections.dot(cell.axes.T))).dot(cell.reciprocal)